
# 自動微分

:ラベル: `sec_autograd`

 :numref: `sec_calculus`を思い出してください。導関数の計算は、ディープ ネットワークのトレーニングに使用するすべての最適化アルゴリズムにおいて重要なステップです。計算は簡単ですが、手作業で計算するのは面倒で間違いが発生しやすく、モデルが複雑になるにつれてこの問題はさらに大きくなります。

幸いなことに、最新のディープ ラーニング フレームワークはすべて*、自動微分*( *autograd*と短縮されることがよくあります) を提供することで、この作業を軽減します。連続する各関数にデータを渡すと、フレームワークは各値が他の値にどのように依存するかを追跡する*計算グラフ*を構築します。導関数を計算するために、自動微分は連鎖規則を適用してこのグラフを逆方向に実行します。この方法で連鎖規則を適用する計算アルゴリズムは、*バックプロパゲーション*と呼ばれます。

 autograd ライブラリは過去 10 年間で大きな関心事になっていますが、その歴史は長いです。実際、autograd への最初の言及は半世紀以上前に遡ります:cite: `Wengert.1964` 。現代のバックプロパゲーションの背後にある中心的なアイデアは、1980 年の博士論文に遡ります :cite: `Speelpenning.1980` 、1980 年代後半にさらに発展しました :cite: `Griewank.1989` 。バックプロパゲーションは勾配を計算するためのデフォルトの方法となっていますが、これが唯一のオプションではありません。たとえば、Julia プログラミング言語は順伝播を採用しています (cite: `Revels.Lubin.Papamarkou.2016` )。メソッドを検討する前に、まず autograd パッケージをマスターしましょう。


In [1]:
import torch


## シンプルな機能

**(関数 $y = 2\mathbf{x}^{\top}\mathbf{x}$ を列ベクトル $\mathbf{x}$ に関して微分する) ことに興味があると仮定しましょう。** `x`に初期値を割り当てます。


In [2]:
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])


[ **$\mathbf{x}$ に対する $y$ の勾配を計算する前に、それを保存する場所が必要です。** ] 一般に、ディープラーニングでは同じパラメータに関して導関数を何千回または何百万回も連続して計算する必要があり、メモリ不足になる危険性があるため、導関数を取得するたびに新しいメモリを割り当てることは避けられます。ベクトル $\mathbf{x}$ に対するスカラー値関数の勾配はベクトル値であり、$\mathbf{x}$ と同じ形状を持つことに注意してください。


In [3]:
# Can also create x = torch.arange(4.0, requires_grad=True)
x.requires_grad_(True)
x.grad  # The gradient is None by default


(**ここで`x`の関数を計算し、その結果を`y`に割り当てます。** )


In [4]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)


[今度は、 `backward`メソッドを呼び出すことで**、 `x`に対する`y`の勾配を取得できます**。次に、 `x`の`grad`属性を介して勾配にアクセスできます。


In [5]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])


**( $\mathbf{x}$ に対する関数 $y = 2\mathbf{x}^{\top}\mathbf{x}$ の勾配は $4\mathbf{x}$ になるはずであることはすでにわかっています。** ) これで、自動勾配計算と予想される結果が同一であることを確認できます。


In [6]:
x.grad == 4 * x

tensor([True, True, True, True])


[**次に、 `x`の別の関数を計算して、その勾配を取得しましょう。** ] 新しい勾配を記録するときに、PyTorch は勾配バッファーを自動的にリセットしないことに注意してください。代わりに、新しいグラデーションがすでに保存されているグラデーションに追加されます。この動作は、複数の目的関数の合計を最適化する場合に便利です。グラデーション バッファーをリセットするには、次のように`x.grad.zero()`を呼び出します。


In [7]:
x.grad.zero_()  # Reset the gradient
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])


## 非スカラー変数の逆方向

`y`がベクトルの場合、ベクトル`x`に関する`y`の導関数の最も自然な解釈は、 `x`の各成分に関する`y`の各成分の偏導関数を含む*ヤコビアン*と呼ばれる行列です。同様に、より高次の`y`および`x`の場合、微分の結果はさらに高次のテンソルになる可能性があります。

ヤコビアンは一部の高度な機械学習手法で使用されますが、より一般的には、完全なベクトル`x`に対する`y`の各成分の勾配を合計して、 `x`と同じ形状のベクトルを生成する必要があります。たとえば、トレーニング サンプルの*バッチ*内のサンプルごとに個別に計算された損失関数の値を表すベクトルがよくあります。ここでは、(**各例に対して個別に計算された勾配を合計する**) ことだけを行います。



深層学習フレームワークは非スカラー テンソルの勾配を解釈する方法が異なるため、PyTorch は混乱を避けるためにいくつかの措置を講じています。オブジェクトをスカラーに減らす方法を PyTorch に指示しない限り、非スカラーで逆方向`backward`呼び出すとエラーが発生します。より正式には、$\partial_ `backward` $\mathbf{v}^\top \partial_{\mathbf{x}} \mathbf{y}$ を逆方向に計算するように、ベクトル $\mathbf{v}$ を提供する必要があります。 {\mathbf{x}} \mathbf{y}$。この次の部分は混乱するかもしれませんが、後で明らかになる理由により、この引数 ($\mathbf{v}$ を表す) には`gradient`という名前が付けられています。より詳細な説明については、Yang Zhang の[Medium 投稿を](https://zhang-yang.medium.com/the-gradient-argument-in-pytorchs-backward-function-explained-by-examples-68f266950c29)参照してください。


In [8]:
x.grad.zero_()
y = x * x
y.backward(gradient=torch.ones(len(y)))  # Faster: y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])


## 計算の切り離し

場合によっては、**一部の計算を記録された計算グラフの外に移動したいことがあります。** ] たとえば、入力を使用して、勾配を計算したくないいくつかの補助中間項を作成するとします。この場合、最終結果からそれぞれの計算グラフ*を切り離す*必要があります。次のおもちゃの例では、これがより明確になります。 `z = x * y`および`y = x * x`があるとしますが、 `y`を介して伝達される影響ではなく、 `x`が`z`に与える*直接的な*影響に焦点を当てたいとします。この場合、 `y`と同じ値を取る新しい変数`u`を作成できますが、その*出所*(作成方法) は消去されています。したがって、グラフ内に`u`は祖先がなく、勾配は`u`を通って`x`に流れません。たとえば、 `z = x * u`の勾配を取得すると、結果`x`が得られます ( `z = x * x * x` `3 * x * x` x * x ではありません)。


In [9]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])


この手順では、 `z`につながるグラフから`y`の祖先が切り離されますが、 `y`につながる計算グラフは存続するため、 `x`に対する`y`の勾配を計算できることに注意してください。


In [10]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])


## グラデーションと Python 制御フロー

これまで、入力から出力へのパスが`z = x * x * x`などの関数によって明確に定義されているケースを検討してきました。プログラミングにより、結果の計算方法がより自由になります。たとえば、補助変数や中間結果の条件選択に依存させることができます。自動微分を使用する利点の 1 つは、(**関数が Python 制御フローの迷路を通過する必要がある**) (条件分岐、ループ、任意の関数呼び出しなど) の計算グラフを構築する**場合でも**、**勾配を計算できること**です。これ**を**説明するために、 `while`ループの反復回数と`if`ステートメントの評価の両方が入力`a`の値に依存する次のコード スニペットを考えてみましょう。


In [11]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c


以下では、この関数を呼び出して、入力としてランダムな値を渡します。入力は確率変数であるため、計算グラフがどのような形式になるかはわかりません。ただし、特定の入力に対して`f(a)`を実行するたびに、特定の計算グラフが実現され、その後`backward`実行できます。


In [12]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()


関数`f`はデモンストレーションの目的で少し工夫されていますが、入力への依存性は非常に単純です。これは区分的に定義されたスケールを持つ`a`の*線形*関数です。したがって、 `f(a) / a`定数エントリのベクトルであり、さらに、 `f(a) / a` `a`に関する`f(a)`の勾配と一致する必要があります。


In [13]:
a.grad == d / a

tensor(True)


動的制御フローはディープラーニングでは非常に一般的です。たとえば、テキストを処理する場合、計算グラフは入力の長さに依存します。このような場合、事前に勾配を計算することは不可能であるため、統計モデリングには自動微分が不可欠になります。

## 議論

これで、自動微分の威力を体験できました。導関数を自動的かつ効率的に計算するためのライブラリの開発は、深層学習の実践者にとって生産性を大幅に向上させ、より高度な関心事に集中できるようになりました。さらに、autograd を使用すると、ペンと紙の勾配計算では法外に時間がかかる大規模なモデルを設計できます。興味深いことに、モデルを (統計的な意味で)*最適化する*ために autograd を使用しますが、autograd ライブラリ自体の (計算的な意味で) の*最適化は*、フレームワーク設計者にとって非常に興味深いテーマです。ここでは、コンパイラのツールとグラフ操作を利用して、最も効率的かつメモリ効率の高い方法で結果を計算します。

ここでは、次の基本を覚えておいてください。(i) 導関数が必要な変数に勾配を付加します。 (ii) 目標値の計算を記録する。 (iii) バックプロパゲーション関数を実行します。 (iv) 結果として得られる勾配にアクセスします。

## 演習
1. 二次導関数は一次導関数よりも計算コストがはるかに高いのはなぜですか?
1. バックプロパゲーションの関数を実行した後、すぐにもう一度実行して、何が起こるかを確認します。なぜ？
1.  `a`に関する`d`の導関数を計算する制御フローの例で、変数`a`をランダム ベクトルまたは行列に変更するとどうなるでしょうか?この時点で、計算結果`f(a)`はスカラーではなくなります。結果はどうなるのでしょうか？これをどのように分析すればよいでしょうか?
1.  $f(x) = \sin(x)$ とします。 $f$ とその導関数 $f&#39;$ のグラフをプロットします。 $f&#39;(x) = \cos(x)$ という事実を利用せず、自動微分を使用して結果を取得します。
1.  $f(x) = ((\log x^2) \cdot \sin x) + x^{-1}$ とします。 $x$ から $f(x)$ までのトレース結果を依存関係グラフに書き出します。
1. 連鎖規則を使用して、前述の関数の導関数 $\frac{df}{dx}$ を計算し、前に作成した依存関係グラフに各項を配置します。
1. グラフと中間導関数の結果が与えられると、勾配を計算するときに多くのオプションがあります。 $x$ から $f$ までを 1 回、$f$ から $x$ まで遡って 1 回、結果を評価します。 $x$ から $f$ へのパスは一般に*前方微分*として知られ、$f$ から $x$ へのパスは後方微分として知られています。
1. どのような場合に前方微分を使用し、どのような場合に後方微分を使用する必要がありますか?ヒント: 必要な中間データの量、ステップを並列化する機能、関連する行列とベクトルのサイズを考慮してください。



[ディスカッション](https://discuss.d2l.ai/t/35)
